# Tutorial 03: road compliance checking

The collision checker library provides a convenient way to construct the road boundary for a scenario and check if the vehicle is within the road.


## 1 Load the scenario

In [ ]:
%matplotlib inline
# commonroad
from commonroad.geometry.shape import Polygon, ShapeGroup, Shape, Rectangle
from commonroad.scenario.lanelet import LaneletNetwork
from commonroad.scenario.obstacle import StaticObstacle, ObstacleType
from commonroad.scenario.scenario import Scenario
from commonroad.scenario.trajectory import State
from commonroad.common.file_reader import CommonRoadFileReader

#commonroad_dc

from commonroad_dc.collision.visualization.draw_dispatch import draw_object
from commonroad_dc.collision.collision_detection.pycrcc_collision_dispatch import create_collision_checker, create_collision_object
import commonroad_dc.pycrcc as pycrcc
from commonroad_dc.boundary import boundary
from commonroad_dc.collision.trajectory_queries import trajectory_queries
from commonroad_dc.pycrcc.Util import trajectory_enclosure_polygons_static


import numpy as np
from time import time


import matplotlib.pyplot as plt

def open_scenario(scenario_filename):

    crfr = CommonRoadFileReader(
        scenario_filename)
    scenario, planning_problem_set = crfr.open()
    return scenario, planning_problem_set

#open the example scenario
scenario, planning_problem_set = open_scenario("Tut_compliance.xml")


# plot the scenario
plt.figure(figsize=(25, 10))
draw_object(scenario)
draw_object(planning_problem_set)
plt.autoscale()
plt.gca().set_aspect('equal')
plt.show()


## 2 Build the road boundary

### 2.1.1 Triangles

In [ ]:
time1=time()
road_boundary_obstacle, road_boundary_sg_triangles=boundary.create_road_boundary_obstacle(scenario, method='triangulation')
time2=time()

print(time2-time1)

# draw the road boundary
plt.figure(figsize=(25, 10))
draw_object(road_boundary_sg_triangles)
plt.autoscale()
plt.gca().set_aspect('equal')
plt.show()
print(road_boundary_sg_triangles.size())



### 2.1.2 Axis-Aligned Triangles (Horizontal)

In [ ]:


time1=time()
road_boundary_obstacle, road_boundary_sg_aligned_triangles=boundary.create_road_boundary_obstacle(scenario, method='aligned_triangulation', axis=1)
time2=time()

print(time2-time1)

# draw the road boundary
plt.figure(figsize=(25, 10))
draw_object(road_boundary_sg_aligned_triangles)
plt.autoscale()
plt.gca().set_aspect('equal')
plt.show()

print(road_boundary_sg_aligned_triangles.size())



### 2.1.3 Axis-Aligned Triangles (Vertical)

In [ ]:


time1=time()
road_boundary_obstacle, road_boundary_sg_aligned_triangles=boundary.create_road_boundary_obstacle(scenario, method='aligned_triangulation', axis=2)
time2=time()

print(time2-time1)

# draw the road boundary
plt.figure(figsize=(25, 10))
draw_object(road_boundary_sg_aligned_triangles)
plt.autoscale()
plt.gca().set_aspect('equal')
plt.show()

print(road_boundary_sg_aligned_triangles.size())



### 2.2 Rectangles

In [ ]:
time1=time()
_, road_boundary_sg_rectangles=boundary.create_road_boundary_obstacle(scenario, method='obb_rectangles')
time2=time()

print(time2-time1)

# draw the road boundary
plt.figure(figsize=(25, 10))
draw_object(road_boundary_sg_rectangles)
plt.autoscale()
plt.gca().set_aspect('equal')
plt.show()

print(road_boundary_sg_rectangles.size())


### 2.3.1 Lane polygons

In [ ]:
time1=time()
road_boundary_sg_polygons=boundary.create_road_polygons(scenario, method='lane_polygons',buffer=1,resample=1, triangulate=False)
time2=time()

print(time2-time1)
# draw the road boundary
plt.figure(figsize=(25, 10))
draw_object(road_boundary_sg_polygons)
plt.autoscale()
plt.gca().set_aspect('equal')
plt.show()


### 2.3.2 Polygons with Holes (Grid)

In [ ]:

# length of the ego-vehicle
box_length=scenario.dynamic_obstacles[0].prediction.shape.length/2
# width of the ego-vehicle
box_width=scenario.dynamic_obstacles[0].prediction.shape.width/2

time1=time()
road_boundary_sg_polygons_grid=boundary.create_road_polygons(scenario, method='whole_polygon_tiled', max_cell_width=box_length*3, max_cell_height=box_length*5,triangulate=False)
time2=time()

print(time2-time1)
# draw the road boundary
plt.figure(figsize=(25, 10))
draw_object(road_boundary_sg_polygons_grid)
#for poly in road_boundary_sg_polygons.unpack():
#    draw_object(poly.triangle_mesh())

plt.autoscale()
plt.gca().set_aspect('equal')
plt.show()

### 2.3.3 Polygon with Holes

In [ ]:
time1=time()
road_boundary_sg_polygons=boundary.create_road_polygons(scenario, method='whole_polygon', triangulate=False)
time2=time()

print(time2-time1)
# draw the road boundary
plt.figure(figsize=(25, 10))
draw_object(road_boundary_sg_polygons)
plt.autoscale()
plt.gca().set_aspect('equal')
plt.show()

## Check if the trajectory is within the road

### Select part of the trajectory for compliance checks

In [ ]:
import commonroad_dc.pycrcc as pycrcc

start_step=3

# length of the ego-vehicle
box_length=scenario.dynamic_obstacles[0].prediction.shape.length/2

# width of the ego-vehicle
box_width=scenario.dynamic_obstacles[0].prediction.shape.width/2

#list with trajectory states that are checked for collisions with the road boudnary
traj_list=list()
state_list=scenario.dynamic_obstacles[0].prediction.trajectory.state_list

# add all trajectory steps from Step 2 onwards to traj_list
for el in state_list[start_step:]:

    new_el=list()
    new_el.append(el.position[0])
    new_el.append(el.position[1])
    new_el.append(el.orientation)
    traj_list.append(new_el)
    
#draw the ego-vehicle at Step i of the traj_list
i=0
obb=pycrcc.RectOBB(box_length,box_width , traj_list[i][2],traj_list[i][0],traj_list[i][1])
obb_next=pycrcc.RectOBB(box_length,box_width , traj_list[i+1][2],traj_list[i+1][0],traj_list[i+1][1])

obb_merged=obb.merge(obb_next)


# draw the road boundary
plt.figure(figsize=(25, 25))
draw_object(road_boundary_sg_triangles)
#draw_object(road_boundary_sg_polygons)


draw_object(obb)
draw_object(obb_next)
draw_object(obb_merged)

#plt.ylim(-7,12)
#plt.xlim(25,75)

plt.autoscale()
plt.gca().set_aspect('equal')

plt.show()


In [ ]:
def create_tvobstacle(traj_list,box_length,box_width):
    retObj=pycrcc.TimeVariantCollisionObject(0)
    for time_step in traj_list:
        retObj.append_obstacle(pycrcc.RectOBB(box_length, box_width, time_step[2],time_step[0],time_step[1]))
    return retObj    
    
    
co=create_tvobstacle(traj_list,box_length,box_width)

In [ ]:
preprocessed_trajectories=list()
trajectories_first_object=list()
for el in range(1000):
    trajectories_first_object.append(traj_list[start_step:start_step])
cur_time_1=time()
for el in range(1000):
    traj, err=trajectory_queries.trajectory_preprocess_obb_sum(co)
    if(err):
        raise Exception("trajectory preprocessing error")
    preprocessed_trajectories.append(traj)

cur_time_2 = time()

print('trajectory preprocessing time (OBB sum hull) for 1000 trajectories: ' + str(cur_time_2-cur_time_1))

### Method 1: Collision Checks - triangles

In [ ]:
num_trials=10

from time import time
cur_time_1=time()
for i in range(num_trials):
    candidate_trajectories=list()
    for ind,el in enumerate(trajectories_first_object):
        if trajectory_enclosure_polygons_static(road_boundary_sg_polygons,box_length, box_width, el)==-1:    
            candidate_trajectories.append(preprocessed_trajectories[ind])
    ret=trajectory_queries.trajectories_collision_static_obstacles(candidate_trajectories, road_boundary_sg_aligned_triangles, method='grid', num_cells=32, auto_orientation=True)

    
cur_time_2 = time()
print("Time for 1000 trajectory checks: " + str((cur_time_2-cur_time_1)/num_trials)+ " sec.")


### Method 2: Polygon enclosure checks

In [ ]:
num_trials=10

from time import time
cur_time_1=time()
for i in range(num_trials):
    candidate_trajectories=list()
    for ind,el in enumerate(preprocessed_trajectories):
        candidate_trajectories.append(el)
    ret=trajectory_queries.trajectories_enclosure_polygons_static(candidate_trajectories, road_boundary_sg_polygons_grid, method='grid', num_cells=32, enable_verification=False)

cur_time_2 = time()
print("Time for 1000 trajectory checks: " + str((cur_time_2-cur_time_1)/num_trials)+ " sec.")

### Method 3: OBB rectangle boundary

In [ ]:
num_trials=10

from time import time
cur_time_1=time()
for i in range(num_trials):
    candidate_trajectories=list()
    for ind,el in enumerate(trajectories_first_object):
        if trajectory_enclosure_polygons_static(road_boundary_sg_polygons,box_length, box_width, el)==-1:    
            candidate_trajectories.append(preprocessed_trajectories[ind])
    ret=trajectory_queries.trajectories_collision_static_obstacles(candidate_trajectories, road_boundary_sg_rectangles, method='grid', num_cells=32, auto_orientation=True)

    
cur_time_2 = time()
print("Time for 1000 trajectory checks: " + str((cur_time_2-cur_time_1)/num_trials)+ " sec.")
